In [1]:
from importlib import reload
import torch
import torch.nn.functional as F
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score

import data
reload(data)
from data import AmlsimDataset

import modules
reload(modules)
from modules import GCN, GCN_GNNExplainer, GCN_GraphSVX
from modules import GraphSAGE
from torch_geometric.data import DataLoader
import torch.optim as optim


In [2]:
# Set device to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cuda


In [3]:
# Load data
traindata = AmlsimDataset(node_file='data/simulation2/swedbank/train/nodes.csv', edge_file='data/simulation2/swedbank/train/edges.csv', node_features=True, node_labels=True).get_data()
testdata = AmlsimDataset(node_file='data/simulation2/swedbank/test/nodes.csv', edge_file='data/simulation2/swedbank/test/edges.csv', node_features=True, node_labels=True).get_data()
traindata = traindata.to(device)
testdata = testdata.to(device)

# # Convert label tensors to one-hot encoded form
# traindata.y = F.one_hot(traindata.y, num_classes=2)
# testdata.y = F.one_hot(testdata.y, num_classes=2)

In [4]:
# Normalize data
mean = traindata.x.mean(dim=0, keepdim=True)
std = traindata.x.std(dim=0, keepdim=True)
traindata.x = (traindata.x - mean) / std
testdata.x = (testdata.x - mean) / std

In [5]:
# Instantiate model
input_dim = 10
hidden_dim = 16
output_dim = 2
n_layers = 3
dropout = 0.3
model = GCN_GraphSVX(input_dim, hidden_dim, output_dim, n_layers, dropout)
model.to(device)

GCN_GraphSVX(
  (convs): ModuleList(
    (0): GCNConv(10, 16)
    (1): GCNConv(16, 16)
    (2): GCNConv(16, 2)
  )
  (bns): ModuleList(
    (0-1): 2 x BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (softmax): Softmax(dim=1)
)

In [6]:
print(model.output_dim)

2


In [7]:
# optimizer
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [8]:
# loss function
criterion = torch.nn.NLLLoss()

In [9]:
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model.forward(traindata.x, traindata.edge_index)
    loss = criterion(out, traindata.y)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        model.eval()
        with torch.no_grad():
            out = model.forward(testdata.x, testdata.edge_index)
            loss = criterion(out, testdata.y)
            precision = precision_score(testdata.y.cpu().numpy(), out.cpu().numpy().argmax(axis=1), zero_division=0)
            recall = recall_score(testdata.y.cpu().numpy(), out.cpu().numpy().argmax(axis=1), zero_division=0)
            print(f'epoch: {epoch + 1}, loss: {loss:.4f}, precision: {precision:.4f}, recall: {recall:.4f}')

epoch: 10, loss: -0.5064, precision: 0.3680, recall: 0.2771
epoch: 20, loss: -0.5379, precision: 0.5000, recall: 0.1928
epoch: 30, loss: -0.5867, precision: 0.6047, recall: 0.1566
epoch: 40, loss: -0.6238, precision: 0.7714, recall: 0.1627
epoch: 50, loss: -0.6495, precision: 0.8378, recall: 0.1867
epoch: 60, loss: -0.6695, precision: 0.8378, recall: 0.1867
epoch: 70, loss: -0.6855, precision: 0.8333, recall: 0.1807
epoch: 80, loss: -0.6970, precision: 0.8333, recall: 0.1807
epoch: 90, loss: -0.7066, precision: 0.8571, recall: 0.1807
epoch: 100, loss: -0.7138, precision: 0.8571, recall: 0.1807
epoch: 110, loss: -0.7189, precision: 0.8571, recall: 0.1807
epoch: 120, loss: -0.7229, precision: 0.8571, recall: 0.1807
epoch: 130, loss: -0.7261, precision: 0.8788, recall: 0.1747
epoch: 140, loss: -0.7283, precision: 0.8788, recall: 0.1747
epoch: 150, loss: -0.7298, precision: 0.8710, recall: 0.1627
epoch: 160, loss: -0.7318, precision: 0.8750, recall: 0.1687
epoch: 170, loss: -0.7333, precis

In [10]:
from sklearn.metrics import confusion_matrix
import numpy as np

model.eval()
with torch.no_grad():
    out = model.forward(testdata.x, testdata.edge_index)
    y_pred = out.cpu().numpy().argmax(axis=1)
    y_true = testdata.y.cpu().numpy()
    cm = confusion_matrix(y_true, y_pred)
    print(cm)





[[394   6]
 [138  28]]


In [11]:
from sklearn.model_selection import train_test_split

def split_function(y, args_train_ratio=0.6, seed=10):
    return _get_train_val_test_masks(y.shape[0], y, (1-args_train_ratio)/2, (1-args_train_ratio), seed=seed)

def _get_train_val_test_masks(total_size, y_true, val_fraction, test_fraction, seed):
    """Performs stratified train/test/val split

    Args:
        total_size (int): dataset total number of instances
        y_true (numpy array): labels
        val_fraction (int): validation/test set proportion
        test_fraction (int): test and val sets proportion
        seed (int): seed value

    Returns:
        [torch.tensors]: train, validation and test masks - boolean values
    """
    # Split into a train, val and test set
    # Store indexes of the nodes belong to train, val and test set
    indexes = range(total_size)
    indexes_train, indexes_test = train_test_split(
        indexes, test_size=test_fraction, stratify=y_true, random_state=seed)
    indexes_train, indexes_val = train_test_split(indexes_train, test_size=val_fraction, stratify=y_true[indexes_train],
                                                  random_state=seed)
    # Init masks
    train_idxs = np.zeros(total_size, dtype=bool)
    val_idxs = np.zeros(total_size, dtype=bool)
    test_idxs = np.zeros(total_size, dtype=bool)

    # Update masks using corresponding indexes
    train_idxs[indexes_train] = True
    val_idxs[indexes_val] = True
    test_idxs[indexes_test] = True

    return torch.from_numpy(train_idxs), torch.from_numpy(val_idxs), torch.from_numpy(test_idxs)

In [12]:
from types import SimpleNamespace

train_ratio = 0.8
testdata.to('cpu')
model.to('cpu')

data = SimpleNamespace()
data.x = testdata.x
data.edge_index  = testdata.edge_index
data.y = testdata.y
data.num_classes = 2
data.num_features = 10
data.num_nodes = testdata.x.shape[0]
data.name = 'test'
data.train_mask, data.val_mask, data.test_mask = split_function(data.y.numpy(), train_ratio)

In [13]:
import GraphSVX_explainers
reload(GraphSVX_explainers)

from GraphSVX_explainers import GraphSVX, GraphLIME


explainer = GraphSVX(data, model, True)

In [17]:
explanations = explainer.explain(node_indexes=range(10,15))

Explainations only consider node features


10it [00:00, 70.13it/s]

WLS: Matrix not invertible
r2:  0.9962832860625991
weighted r2:  0.9999896247863371
Explanations include 10 node features and 0 neighbours for this node        for 2 classes
Model prediction is class 0 with confidence 2.5924456119537354, while true label is 0
Base value 2.646485242351872 for class  0
Weights for node features:  -0.05403806267895561 and neighbours:  0
Most influential features:  [(7, 0.9804937226581387), (6, -0.8046020945766941), (4, -0.3630704879760742)]
Time:  0.1974503993988037
Explainations only consider node features



10it [00:00, 66.19it/s]

WLS: Matrix not invertible
r2:  0.9076794462848671
weighted r2:  0.9999153460950052
Explanations include 9 node features and 0 neighbours for this node        for 2 classes
Model prediction is class 0 with confidence 2.5759079456329346, while true label is 0
Base value 2.646421509595487 for class  0
Weights for node features:  -0.07051832590635199 and neighbours:  0


...

10it [00:00, 61.20it/s]

WLS: Matrix not invertible
r2:  0.9949677113369303
weighted r2:  0.9999828091567446
Explanations include 9 node features and 0 neighbours for this node        for 2 classes


Model prediction is class 0 with confidence 2.666609287261963, while true label is 0
Base value 2.646312785500621 for class  0
Weights for node features:  0.020295840814287924 and neighbours:  0
Most influential features:  [(9, 0.08223654709581751), (8, 0.03657207960714004), (0, -0.03450490057002753)]
Time:  0.8360488414764404
Explainations only consider node features


10it [00:00, 71.09it/s]

WLS: Matrix not invertible
r2:  0.8714810347181036
weighted r2:  0.9995736284195016
Explanations include 9 node features and 0 neighbours for this node        for 2 classes
Model prediction is class 0 with confidence 2.6664745807647705, while true label is 0
Base value 2.6463108725140962 for class  0
Weights for node features:  0.020161201086009495 and neighbours:  0
Most influential features:  [(9, 0.05009955660352716), (7, -0.01732225705540813), (8, -0.010469432938619017)]
Time:  1.0562725067138672
